__Vi bygger om logiken för att plocka ut 24 hästar. Optimerad modell__

In [17]:
# Common imports
import numpy as np
import os
import pandas as pd


# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [18]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [19]:
from DataPipeline import DataFrameSelector
from DataPipeline import SetDummyVar
from sklearn.externals import joblib

__Läser in filen som ska köras__

In [20]:
df1 = pd.read_excel('RealDataV75TillUffeU20190413.xlsx')

In [21]:
df1 = df1.sort_values(['Lopp','Hast'], ascending = ['True','False']).reset_index()

In [22]:
# Importera model som ska användas
rf = joblib.load('Travmodel_v4.pkl')


In [23]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_v4.pkl')

__Definierar de variabler som ska användas i datasetet__

In [24]:
# Kategorivariabel
cat_attribs = ['GRUPP']
# Numerisk variabel 

num_attribs = ['VLP',
 'SVLP',
 'VSVLP',
 'VPN_SUM',
 'VPN_SUM_ORD',
 'VPK_SUM',
 'VPK_SUM_ORD',
 'VLPB',
 'SVLPB',
 'VSVLPB',
 'E_P',
 'E_P_Num',
 'E_N',
 'E_R',
 'E_U',
 'G_R',
 'A_R',
 'T_R',
 'ToR',
 'Ts_R']

__Kör modellen och plockar fram de 24 hästarna med högst score__

In [25]:
df1['GRUPP'] = df1.GRUPP.astype('object')
features = full_pipeline.fit_transform(df1)

predict = rf.predict_proba(features)

predict_cutoff = pd.DataFrame({'Prob0':predict[:,0],'Prob1':predict[:,1]}).sort_values('Prob1', ascending = False).iloc[25]['Prob1']

y_pred = np.where( predict[:,1] > predict_cutoff,1,0)

In [26]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [27]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [28]:
# Gör om till lista och stoppar in i ett dictionary
pred = y_pred.tolist()
prediction = {'Predict':pred}
# Konverterar till Dataframe
df_pred = pd.DataFrame.from_dict(prediction)

In [29]:
df_predict1 = df1.merge(df_pred, right_index = True, left_index = True)
df_predict2 = df_predict1.merge(df_scored,  right_index = True, left_index = True) 

In [30]:
df1_out = df_predict2[df_predict2.Predict == 1][['Lopp','Hast','Score']]. \
sort_values(['Lopp','Score'], ascending = [True,False])
df1_out

,Lopp,Hast,Score
4,1,5,0.472203
2,1,3,0.155234
7,1,9,0.137876
8,2,2,0.215131
16,2,10,0.138293
9,2,3,0.132020
18,2,12,0.094686
23,3,5,0.823243
20,3,2,0.153389
21,3,3,0.093294


In [31]:
len(df1_out)

25

__Export till excel__

In [32]:
df1_out.to_excel('Outtake20190413.xlsx', index = False)